In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy import data
import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
!pip install spacy --upgrade

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)


In [3]:
%%bash
python -m spacy download en
# python -m spacy download de

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-19 09:35:34.892478: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
# spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

## DataSet from http://www.cs.cmu.edu/~ark/QA-data/

In [5]:
!wget http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz

--2021-06-19 09:02:54--  http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8254496 (7.9M) [application/x-gzip]
Saving to: ‘Question_Answer_Dataset_v1.2.tar.gz’

Question_Answer_Dat 100%[===================>]   7.87M  3.80MB/s    in 2.1s    

2021-06-19 09:02:56 (3.80 MB/s) - ‘Question_Answer_Dataset_v1.2.tar.gz’ saved [8254496/8254496]



In [6]:
! tar -xf /content/Question_Answer_Dataset_v1.2.tar.gz

In [5]:
import glob

import pandas as pd


QA_dataset_complete = pd.DataFrame()

# get all .txt files from all subdirectories

all_files = glob.glob('./Question_Answer_Dataset_v1.2/*/*.txt')


for file in all_files:

  QA_dataset = pd.read_csv(file, sep='\t', encoding= 'ISO-8859-1')
                           #'unicode_escape',header=None,error_bad_lines=False,skiprows=[0])
  print(f'Complete file path :{file} | Number of records or dimentions : {QA_dataset.shape}')
  QA_dataset_complete=pd.concat([QA_dataset_complete,QA_dataset])


print(f'After combining all files total number of records or dimentions : {QA_dataset_complete.shape}')

Complete file path :./Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt | Number of records or dimentions : (1715, 6)
Complete file path :./Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt | Number of records or dimentions : (1458, 6)
Complete file path :./Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt | Number of records or dimentions : (825, 6)
After combining all files total number of records or dimentions : (3998, 6)


In [6]:
QA_dataset_complete.head(10)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,data/set3/a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,data/set3/a4
5,Abraham_Lincoln,Did his mother die of pneumonia?,No.,easy,easy,data/set3/a4
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months,medium,easy,data/set3/a4
7,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months.,medium,medium,data/set3/a4
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832,medium,easy,data/set3/a4
9,Abraham_Lincoln,When did Lincoln begin his political career?,1832.,medium,medium,data/set3/a4


In [7]:
QA_dataset_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998 entries, 0 to 824
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              3998 non-null   object
 1   Question                  3961 non-null   object
 2   Answer                    3422 non-null   object
 3   DifficultyFromQuestioner  3043 non-null   object
 4   DifficultyFromAnswerer    3418 non-null   object
 5   ArticleFile               3996 non-null   object
dtypes: object(6)
memory usage: 218.6+ KB


In [8]:
QA_dataset_complete.describe()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
count,3998,3961,3422,3043,3418,3996
unique,109,2456,1828,4,5,57
top,Amedeo_Avogadro,Was King Victor Emmanuel III there to pay homa...,Yes,easy,easy,data/set4/a8
freq,132,6,492,1035,1344,132


In [9]:
QA_dataset_complete.isnull().sum()

ArticleTitle                  0
Question                     37
Answer                      576
DifficultyFromQuestioner    955
DifficultyFromAnswerer      580
ArticleFile                   2
dtype: int64

In [10]:
QA_dataset_complete.dropna(subset=['Question','Answer'], inplace=True)
QA_dataset_complete.shape

(3422, 6)

In [11]:
QA_dataset_complete.isnull().sum()

ArticleTitle                  0
Question                      0
Answer                        0
DifficultyFromQuestioner    688
DifficultyFromAnswerer        5
ArticleFile                   2
dtype: int64

In [12]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [13]:
question = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

answer = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

fields = [('questions', question),('answers',answer)]

In [14]:
QA_dataset_complete.reset_index(drop=True,  inplace=True)

In [15]:
QA_dataset_examples = [data.Example.fromlist([str(QA_dataset_complete.Question[i]),str(QA_dataset_complete.Answer[i])],fields) for i in range(QA_dataset_complete.shape[0])]

In [16]:
QA_Dataset= data.Dataset(QA_dataset_examples, fields)

In [17]:
(train_data, test_data) = QA_Dataset.split(split_ratio=[0.7,0.3], random_state=random.seed(SEED)) 

In [18]:
print(f"Toatal examples: {len(QA_Dataset.examples)}")
print(f"Number of training examples: {len(train_data.examples)}")
# print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Toatal examples: 3422
Number of training examples: 2395
Number of testing examples: 1027


In [19]:
print(vars(train_data.examples[0]))

{'questions': ['what', 'was', 'the', 'faraday', 'effect', 'first', 'called', '?'], 'answers': ['diamagnetism']}


In [36]:
print(vars(test_data.examples[0]))

{'questions': ['where', 'is', 'the', 'most', 'densely', 'populated', 'part', 'of', 'canada', '?'], 'answers': ['the', 'most', 'densely', 'populated', 'part', 'of', 'the', 'country', 'is', 'the', 'quebec', 'city', '-', 'windsor', 'corridor', 'along', 'the', 'great', 'lakes', 'and', 'saint', 'lawrence', 'river', 'in', 'the', 'southeast', '.']}


In [20]:
question.build_vocab(train_data, min_freq = 2)
answer.build_vocab(train_data, min_freq = 2)

In [21]:
print(f"Unique tokens in source vocabulary: {len(question.vocab)}")
print(f"Unique tokens in target vocabulary: {len(answer.vocab)}")

Unique tokens in source vocabulary: 2024
Unique tokens in target vocabulary: 1282


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key= lambda x: len(x.questions),
    sort_within_batch=True, 
    device = device)

In [43]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [44]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [45]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [46]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [47]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2024, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1282, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1282, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,860,418 trainable parameters


In [49]:
optimizer = optim.Adam(model.parameters())

In [50]:
TRG_PAD_IDX = answer.vocab.stoi[answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [51]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions
        trg = batch.answers
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [52]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions
            trg = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [53]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [54]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 5.183 | Train PPL: 178.164
	 Val. Loss: 3.600 |  Val. PPL:  36.602
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.406 | Train PPL:  81.964
	 Val. Loss: 3.614 |  Val. PPL:  37.101
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.324 | Train PPL:  75.460
	 Val. Loss: 3.514 |  Val. PPL:  33.595
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.246 | Train PPL:  69.828
	 Val. Loss: 3.491 |  Val. PPL:  32.818
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.182 | Train PPL:  65.481
	 Val. Loss: 3.457 |  Val. PPL:  31.710
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.081 | Train PPL:  59.220
	 Val. Loss: 3.452 |  Val. PPL:  31.548
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.011 | Train PPL:  55.197
	 Val. Loss: 3.649 |  Val. PPL:  38.448
Epoch: 08 | Time: 0m 2s
	Train Loss: 3.959 | Train PPL:  52.417
	 Val. Loss: 3.366 |  Val. PPL:  28.961
Epoch: 09 | Time: 0m 2s
	Train Loss: 3.925 | Train PPL:  50.656
	 Val. Loss: 3.393 |  Val. PPL:  29.755
Epoch: 10 | Time: 0m 2s
	Train Loss: 3.881 | Train PPL:  48.491


In [55]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.366 | Test PPL:  28.961 |
